# モデル学習

Hugging Face Transformersを使用してモデルを学習します。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/)

## 1. 環境設定

まず、GPUが有効になっているか確認します。

**設定方法**: メニュー → ランタイム → ランタイムのタイプを変更 → GPU を選択

In [ ]:
# GPU確認
import torch
print(f"GPU利用可能: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")

## 2. ライブラリのインストール

In [ ]:
!pip install -q transformers datasets evaluate accelerate

## 3. ライブラリのインポート

In [ ]:
from typing import Optional, Dict, Any
from datasets import DatasetDict
from transformers import (
    PreTrainedModel,
    PreTrainedTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
import evaluate
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 4. 学習関数の定義

In [ ]:
def train(
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizer,
    dataset: DatasetDict,
    output_dir: str = "./results",
    num_train_epochs: int = 3,
    per_device_train_batch_size: int = 16,
    per_device_eval_batch_size: int = 16,
    warmup_steps: int = 500,
    weight_decay: float = 0.01,
    logging_dir: str = "./logs",
    logging_steps: int = 10,
    eval_strategy: str = "epoch",
    save_strategy: str = "epoch",
    load_best_model_at_end: bool = True,
    metric_for_best_model: str = "accuracy",
    greater_is_better: bool = True,
    save_total_limit: int = 3,
    fp16: bool = True,
    gradient_accumulation_steps: int = 1,
    learning_rate: float = 5e-5,
    lr_scheduler_type: str = "linear",
    seed: int = 42,
    data_seed: int = 42,
    **kwargs
) -> Trainer:
    """
    モデルを学習する関数

    Args:
        model: 学習対象のモデル
        tokenizer: トークナイザー
        dataset: 学習データセット（train, validationを含む）
        output_dir: 出力ディレクトリ
        num_train_epochs: エポック数
        per_device_train_batch_size: デバイスごとの訓練バッチサイズ
        per_device_eval_batch_size: デバイスごとの評価バッチサイズ
        warmup_steps: ウォームアップステップ数
        weight_decay: 重み減衰
        logging_dir: ログディレクトリ
        logging_steps: ログ出力ステップ間隔
        eval_strategy: 評価戦略
        save_strategy: 保存戦略
        load_best_model_at_end: 学習終了時にベストモデルをロード
        metric_for_best_model: ベストモデル判定のメトリクス
        greater_is_better: メトリクスが大きい方が良いか
        save_total_limit: 保存するチェックポイントの最大数
        fp16: 半精度学習を使用
        gradient_accumulation_steps: 勾配蓄積ステップ数
        learning_rate: 学習率
        lr_scheduler_type: 学習率スケジューラタイプ
        seed: 乱数シード
        data_seed: データシード
        **kwargs: その他の引数

    Returns:
        Trainer: 学習済みTrainer
    """
    logger.info("Starting training...")
    logger.info(f"Model: {model.__class__.__name__}")
    logger.info(f"Dataset: {dataset}")
    logger.info(f"Output dir: {output_dir}")

    # データコレーターの設定
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # 評価メトリクスの設定
    accuracy = evaluate.load("accuracy")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return accuracy.compute(predictions=predictions, references=labels)

    # 学習設定
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        logging_dir=logging_dir,
        logging_steps=logging_steps,
        eval_strategy=eval_strategy,
        save_strategy=save_strategy,
        load_best_model_at_end=load_best_model_at_end,
        metric_for_best_model=metric_for_best_model,
        greater_is_better=greater_is_better,
        save_total_limit=save_total_limit,
        fp16=fp16,
        gradient_accumulation_steps=gradient_accumulation_steps,
        learning_rate=learning_rate,
        lr_scheduler_type=lr_scheduler_type,
        seed=seed,
        data_seed=data_seed,
        report_to=[],
        **kwargs
    )

    # Trainerの作成
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset.get("validation", dataset.get("test")),
        processing_class=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # 学習実行
    trainer.train()

    logger.info("Training completed")
    return trainer

## 5. 使用例

In [ ]:
# 使用例
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

# モデルとトークナイザーの読み込み
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# データセットの読み込み
dataset = load_dataset("imdb")

# 前処理
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

# 学習実行
trainer = train(
    model=model,
    tokenizer=tokenizer,
    dataset=tokenized_dataset,
    output_dir="./results",
    num_train_epochs=1,  # デモ用に1エポック
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_steps=100,
    save_total_limit=1
)

print("学習完了")
print(f"ベストモデルのメトリクス: {trainer.state.best_metric}")

In [ ]:
# 学習済みモデルのテスト
from transformers import pipeline

# 学習したモデルでpipelineを作成
classifier = pipeline("text-classification", model=trainer.model, tokenizer=tokenizer)

# テストテキスト
test_texts = [
    "This movie is fantastic!",
    "I didn't like this film at all.",
    "It was an average movie."
]

# 推論実行
results = classifier(test_texts)
for text, result in zip(test_texts, results):
    print(f"Text: {text}")
    print(f"Prediction: {result['label']} (confidence: {result['score']:.4f})")
    print()
